In [1]:
import ee
import os

from src.controllers.plotter_base import PhenologyPlotter
from src.controllers.metrics_vos_pos import VosPosMetrics
from src.controllers.metrics_bos_eso import BosEosMetrics
from src.controllers.sentinel_ import  Sentinel2Processor
from src.controllers.geometry import ProcessadorGeoDataFrame
from src.controllers.metrics_geometrics import PhenologyMetrics
from src.controllers.time_series import VegetationIndexProcessor
from src.controllers.eras import PrecipitationTemperatureRadiationData

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-carbonei')

In [2]:
####Geometries variables####

#Paht polygons
path = r"C:\Users\ander\OneDrive\Área de Trabalho\msu\phenologic_metrics\data\ms_field_boundaries_samples.gpkg"

#Index polygon, defoult is 0
index_poligon = 16

####Datas de análise####
#Start date analises 
start_date = '2023-01-01'

#End date analises
end_date = '2023-12-30'


####Time series parameters####
#ndvi_value to NDVI and evi_value to EVI 
vegetation_index = 'ndvi_value'

#Threshould index to NDVI recomend 0.37
threshould_index = 0.3

####Savitzky-Golay to smoothing the vegetation index####
# Window to smooth Savitzky-Golay
window_size = 5

# Type of polynomial 
poly_order = 2

####VOS and POS metric####
#This value define the window to get VOS and POS, by experience use 3 parts os the full crop season
order = 40

In [3]:
#Read geometry file
processador = ProcessadorGeoDataFrame(path)

In [4]:
vertices, geometry = processador.extrair_coordenadas(index_poligon)

In [5]:
def process_polygon(index_poligon,id_polygon, start_date, end_date, vegetation_index, threshould_index, window_size, poly_order, order):

    try:

        vertices, geometry = processador.extrair_coordenadas(index_poligon)

        # Convert points to a polygon
        polygon = ee.Geometry.Polygon(vertices)

        s2 = Sentinel2Processor(start_date, end_date, polygon)
        df = s2.process_data()

        processor = VegetationIndexProcessor(df, vegetation_index, window_size=window_size, poly_order=poly_order)
        df = processor.process()

        #Get and VOS and POS metrics
        vos_pos_analyzer = VosPosMetrics(df , order)
        phenology_df = vos_pos_analyzer.analyze_phenology()

        #Get and BOS and EOS metrics
        analysis = BosEosMetrics(df , phenology_df, threshould_index)
        phenology_df  = analysis.execute_analysis()

        analysis_metrics = PhenologyMetrics(phenology_df, df)
        phenology_df  = analysis_metrics.derivate_metrics()

        # Usage
        lat = geometry.getInfo()['coordinates'][0][0][1]  # Exemplo de latitude
        lon = geometry.getInfo()['coordinates'][0][0][0]  # Exemplo de longitude

        data = PrecipitationTemperatureRadiationData(lat, lon, start_date, end_date)
        df_01 = data.get_dataframe()

        df_01 = data.preciptation_sum(phenology_df)

        plotter = PhenologyPlotter(df, phenology_df, vegetation_index)
        #fig = plotter.plot_data()

        #fig_2 = plotter.plot_data_02(start_date, end_date)

        #fig_precipitation = data.plot_precipitation()
        #fig_temperature = data.plot_temperature()
        #fig_radiation = data.plot_radiation()


        #fig.show()
        #fig_2.show()
        #   Show the interactive plots
        #fig_precipitation.show()
        #fig_temperature.show()
        #fig_radiation.show()

        # Define the directory name based on the polygon index
        dir_name = f"base/polygon_{id_polygon}"

        # Create the directory if it doesn't exist
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)

        # Define file paths
        phenology_df_path = os.path.join(dir_name, "phenology_df.csv")
        df_01_path = os.path.join(dir_name, "data_base.csv")
        df_path = os.path.join(dir_name, "data_index.csv")

        # Save the dataframes as CSV files
        phenology_df.to_csv(phenology_df_path, index=False)
        df_01.to_csv(df_01_path, index=False)
        df.to_csv(df_path, index=False)

        print(f"Saved phenology_df to {phenology_df_path}")
        print(f"Saved df_01 to {df_01_path}")

    except:
        pass

In [6]:
import geopandas as gpd

# Carregar o arquivo em um GeoDataFrame e ajustar o CRS
gdf = gpd.read_file(path)
gdf = gdf.explode(ignore_index=True)
gdf = gdf.to_crs("epsg:4326")

for index, row in gdf.iterrows():

    print(row['id'], index)

    process_polygon(index,row['id'],start_date, end_date, vegetation_index, threshould_index, window_size, poly_order, order)
    
    

332 0


c:\Users\ander\OneDrive\Área de Trabalho\msu\phenologic_metrics\src\controllers\metrics_bos_eso.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.interval_df['first_derivative'] = self.interval_df['savitzky_golay'].diff().fillna(0)


Saved phenology_df to base/polygon_332\phenology_df.csv
Saved df_01 to base/polygon_332\data_base.csv
333 1


c:\Users\ander\OneDrive\Área de Trabalho\msu\phenologic_metrics\src\controllers\metrics_bos_eso.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.interval_df['first_derivative'] = self.interval_df['savitzky_golay'].diff().fillna(0)


Saved phenology_df to base/polygon_333\phenology_df.csv
Saved df_01 to base/polygon_333\data_base.csv
334 2


...

Saved phenology_df to base/polygon_1327\phenology_df.csv
Saved df_01 to base/polygon_1327\data_base.csv
1335 301


c:\Users\ander\OneDrive\Área de Trabalho\msu\phenologic_metrics\src\controllers\metrics_bos_eso.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.interval_df['first_derivative'] = self.interval_df['savitzky_golay'].diff().fillna(0)


Saved phenology_df to base/polygon_1335\phenology_df.csv
Saved df_01 to base/polygon_1335\data_base.csv
1359 302


c:\Users\ander\OneDrive\Área de Trabalho\msu\phenologic_metrics\src\controllers\metrics_bos_eso.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.interval_df['first_derivative'] = self.interval_df['savitzky_golay'].diff().fillna(0)


Saved phenology_df to base/polygon_1359\phenology_df.csv
Saved df_01 to base/polygon_1359\data_base.csv


In [7]:
import concurrent.futures

# Função para processar polígonos em paralelo
def process_all_polygons(indices):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_polygon, index, start_date, end_date, vegetation_index, threshould_index, window_size, poly_order, order): index for index in indices}
        for future in concurrent.futures.as_completed(futures):
            index = futures[future]
            try:
                fig, fig_2, fig_precipitation, fig_temperature, fig_radiation = future.result()
                # Exibir ou salvar os gráficos aqui, se necessário
            except Exception as exc:
                print(f'Polygon {index} generated an exception: {exc}')
            else:
                print(f'Polygon {index} processed successfully.')

# Execute o processamento paralelo
process_all_polygons([0,1,2])

Polygon 0 generated an exception: A process in the process pool was terminated abruptly while the future was running or pending.
Polygon 1 generated an exception: A process in the process pool was terminated abruptly while the future was running or pending.
Polygon 2 generated an exception: A process in the process pool was terminated abruptly while the future was running or pending.
